Script to produce 3 cleaned SolA datas and 3 GHI datas for certain date and certain site to demonstrate each mode

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import pytz #for timezone calculation
import math
import matplotlib.dates as md
import gc
import os
from datetime import datetime
%matplotlib qt
%matplotlib inline

def input_monthly_files(file_path, data_date_idx):
    """Open time-series D-PV data and ghi data of a certain month. Only compatible for SoLA data format.

    Args:
        file_path (str): The file location of the data
        data_date_idx (str): The month of the files in format 'YYYYMM' eg '201907'

    Returns:
        data (df): the opened time-series D-PV data
        ghi (df): the opened ghi data
        
    Originally written by Baran for VVAr Curtailment Calculation.
    """
    
    data_path = file_path + r"/processed_unsw_" + data_date_idx + '_data_raw.csv'
    data = pd.read_csv(data_path, index_col = 1)

    # Convert timestamp to local Adelaide time
    data.index = pd.to_datetime(data.index) # convert index from object type to datetime
    Adelaide_local_time = pytz.timezone('Australia/Adelaide')
    data.index = data.index.tz_localize(pytz.utc).tz_convert(Adelaide_local_time) # convert utc to local adelaide time
    data.index.rename('Timestamp', inplace = True)

    # Load GHI data
    ghi_date_idx = data_date_idx[0:4] + '_' + data_date_idx[4:]
    ghi_path = file_path + r"/sl_023034_" + ghi_date_idx +'.txt'
    ghi = pd.read_csv (ghi_path) 

    ghi['timestamp'] = pd.to_datetime(pd.DataFrame ({'year' : ghi['Year Month Day Hours Minutes in YYYY'].values, 
                                                    'month' : ghi['MM'], 
                                                    'day' : ghi['DD'], 
                                                   'hour' : ghi['HH24'], 
                                                   'minute' : ghi['MI format in Local standard time']}))
    ghi.set_index('timestamp', inplace = True)
    # Deal with the space characters (ghi is in object/string form at the moment)
    ghi['Mean global irradiance (over 1 minute) in W/sq m'] = [float(ghi_t) if ghi_t.count(' ')<= 3 else np.nan for ghi_t in ghi['Mean global irradiance (over 1 minute) in W/sq m']]
    return data, ghi

file_path = r"C:\Users\samha\Documents\CANVAS\data"
#file_path = r"/Users/samhan/Downloads/data"



In [3]:
#sample_1: tripping_non_clear_sky_sample
c_id = 1317822057
date = '2019-09-03'
month = date[:4] + date[5:7]
data_sample_1, ghi_sample_1 = input_monthly_files(file_path, month)
date_dt = dt.datetime.strptime(date, '%Y-%m-%d').date()

date_filter = data_sample_1.index.date == date_dt
site_id_filter = data_sample_1['c_id'] == c_id
data_sample_1 = data_sample_1.loc[date_filter & site_id_filter]

date_filter = ghi_sample_1.index.date == date_dt
ghi_sample_1 = ghi_sample_1.loc[date_filter]

#vvar_sample
c_id = 1018350709
date = '2019-07-20'
month = date[:4] + date[5:7]
data_sample_2, ghi_sample_2 = input_monthly_files(file_path, month)
date_dt = dt.datetime.strptime(date, '%Y-%m-%d').date()

date_filter = data_sample_2.index.date == date_dt
site_id_filter = data_sample_2['c_id'] == c_id
data_sample_2 = data_sample_2.loc[date_filter & site_id_filter]

date_filter = ghi_sample_2.index.date == date_dt
ghi_sample_2 = ghi_sample_2.loc[date_filter]

#vwatt_sample
c_id = 466930914
date = '2019-08-14'
month = date[:4] + date[5:7]
data_sample_3, ghi_sample_3 = input_monthly_files(file_path, month)
date_dt = dt.datetime.strptime(date, '%Y-%m-%d').date()

date_filter = data_sample_3.index.date == date_dt
site_id_filter = data_sample_3['c_id'] == c_id
data_sample_3 = data_sample_3.loc[date_filter & site_id_filter]

date_filter = ghi_sample_3.index.date == date_dt
ghi_sample_3 = ghi_sample_3.loc[date_filter]



data_sample_1.to_csv(file_path + '/data_sample_1.csv')
data_sample_2.to_csv(file_path + '/data_sample_2.csv')
data_sample_3.to_csv(file_path + '/data_sample_3.csv')


ghi_sample_1.to_csv(file_path + '/ghi_sample_1.csv')
ghi_sample_2.to_csv(file_path + '/ghi_sample_2.csv')
ghi_sample_3.to_csv(file_path + '/ghi_sample_3.csv')


In [6]:
#sample 4: vwatt_sample_json_tim, good for polyfit example
c_id = 1165442853
date = '2020-04-13'
month = date[:4] + date[5:7]
data_sample_4, ghi_sample_4 = input_monthly_files(file_path, month)
date_dt = dt.datetime.strptime(date, '%Y-%m-%d').date()

date_filter = data_sample_4.index.date == date_dt
site_id_filter = data_sample_4['c_id'] == c_id
data_sample_4 = data_sample_4.loc[date_filter & site_id_filter]

date_filter = ghi_sample_4.index.date == date_dt
ghi_sample_4 = ghi_sample_4.loc[date_filter]

data_sample_4.to_csv(file_path + '/data_sample_4.csv')
ghi_sample_4.to_csv(file_path + '/ghi_sample_4.csv')


KeyboardInterrupt



In [3]:
#sample 5: incomplete dataset
sample_num = 5
c_id = 1317822057
date = '2020-03-01'
month = date[:4] + date[5:7]
data_sample, ghi_sample = input_monthly_files(file_path, month)
date_dt = dt.datetime.strptime(date, '%Y-%m-%d').date()

date_filter = data_sample.index.date == date_dt
site_id_filter = data_sample['c_id'] == c_id
data_sample = data_sample.loc[date_filter & site_id_filter]

date_filter = ghi_sample.index.date == date_dt
ghi_sample = ghi_sample.loc[date_filter]

data_sample.to_csv(file_path + '/data_sample_{}.csv'.format(sample_num))
ghi_sample.to_csv(file_path + '/ghi_sample_{}.csv'.format(sample_num))

In [19]:
#sample 6: biggest vvar site, but don't know which date is good. Probably clear sky day in summer, January?
# clear sky day list: '2020-01-01', '2020-01-02', '2020-01-06', '2020-01-07', '2020-01-08', '2020-01-12', '2020-01-13', '2020-01-14', '2020-01-17', '2020-01-29', '2020-01-30'

#this turns out to be a very good vwatt sample
sample_num = 6
c_id = 898652745
date = '2020-01-17'
month = date[:4] + date[5:7]
data_sample, ghi_sample = input_monthly_files(file_path, month)
date_dt = dt.datetime.strptime(date, '%Y-%m-%d').date()

date_filter = data_sample.index.date == date_dt
site_id_filter = data_sample['c_id'] == c_id
data_sample = data_sample.loc[date_filter & site_id_filter]

date_filter = ghi_sample.index.date == date_dt
ghi_sample = ghi_sample.loc[date_filter]

data_sample.to_csv(file_path + '/data_sample_{}.csv'.format(sample_num))
ghi_sample.to_csv(file_path + '/ghi_sample_{}.csv'.format(sample_num))


In [25]:
#sample 7: biggest vvar site according to Baran's script, but don't know which date is good. Probably clear sky day in summer, January?
# clear sky day list: '2020-01-01', '2020-01-02', '2020-01-06', '2020-01-07', '2020-01-08', '2020-01-12', '2020-01-13', '2020-01-14', '2020-01-17', '2020-01-29', '2020-01-30'

#this turns out to be a very good vwatt sample
sample_num = 7
c_id = 1365659170
date = '2020-01-17'
month = date[:4] + date[5:7]
data_sample, ghi_sample = input_monthly_files(file_path, month)
date_dt = dt.datetime.strptime(date, '%Y-%m-%d').date()

date_filter = data_sample.index.date == date_dt
site_id_filter = data_sample['c_id'] == c_id
data_sample = data_sample.loc[date_filter & site_id_filter]

date_filter = ghi_sample.index.date == date_dt
ghi_sample = ghi_sample.loc[date_filter]

data_sample.to_csv(file_path + '/data_sample_{}.csv'.format(sample_num))
ghi_sample.to_csv(file_path + '/ghi_sample_{}.csv'.format(sample_num))


In [26]:
sample1 = pd.read_csv(file_path + '/data_sample_1.csv')
sample2 = pd.read_csv(file_path + '/data_sample_2.csv')
sample3 = pd.read_csv(file_path + '/data_sample_3.csv')
sample4 = pd.read_csv(file_path + '/data_sample_4.csv')
sample5 = pd.read_csv(file_path + '/data_sample_5.csv')
sample6 = pd.read_csv(file_path + '/data_sample_6.csv')
sample7 = pd.read_csv(file_path + '/data_sample_7.csv')

In [27]:
print(len(sample1))
print(len(sample2))
print(len(sample3))
print(len(sample4))
print(len(sample5))
print(len(sample6))
print(len(sample7))

1433
1432
1437
1228
22
1497
17272


In [42]:
sample6

,Timestamp,c_id,energy,power,reactive_power,voltage,duration
0,2020-01-17 05:49:55+10:30,898652745,-113,-1.883333,5998,247.5,60
1,2020-01-17 05:50:55+10:30,898652745,-113,-1.883333,5991,247.2,60
2,2020-01-17 05:51:55+10:30,898652745,-113,-1.883333,5966,247.1,60
3,2020-01-17 05:52:55+10:30,898652745,-113,-1.883333,5986,247.4,60
4,2020-01-17 05:53:55+10:30,898652745,-114,-1.900000,5985,247.0,60
...,...,...,...,...,...,...,...
1492,2020-01-17 11:14:55+10:30,898652745,188223,3137.050000,13260,254.9,60
1493,2020-01-17 11:15:55+10:30,898652745,186894,3114.900000,12935,254.5,60
1494,2020-01-17 11:16:55+10:30,898652745,192079,3201.316667,12256,254.5,60
1495,2020-01-17 11:17:55+10:30,898652745,194096,3234.933333,13615,254.1,60
